In [1]:
import copy
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np

np.set_printoptions(precision=4, suppress=True)
pd.set_option('display.float_format', lambda x: '%.4f' % x)

In [26]:
df = pd.read_csv('../data/ReducedLoanData.csv', low_memory=False)
# if df['Gender'].isna().sum() != 0:
#     value_counts = df['Gender'].value_counts()
#     values = list(set(df['Gender'].unique()) - {np.nan})
#     total_count = sum([value_counts[v] for v in values])
#     value_probabilities = [value_counts[v] / total_count for v in values]
#     nan_count = df['Gender'].isna().sum()
#     df.loc[df['Gender'].isna(), 'Gender'] = np.random.choice(values, size=nan_count, p=value_probabilities)
df

,Default,Age,AppliedAmount,Country,DebtToIncome,Education,EmploymentDurationCurrentEmployer,EmploymentStatus,FreeCash,Gender,HomeOwnershipType,IncomeTotal,LiabilitiesTotal,LoanDuration,MaritalStatus,NewCreditCustomer,NrOfDependants,OccupationArea,UseOfLoan,WorkExperience
0,True,53,2125.0000,EE,26.2900,Secondary,MoreThan5Years,Unemployed,10.9200,Female,Owner,354.0000,485.0900,60,Cohabitant,False,0.0000,Other,Financial,MoreThan15Years
1,True,50,3000.0000,EE,30.5800,Higher,MoreThan5Years,Self-employed,78.8000,Female,Owner,900.0000,736.4500,60,Cohabitant,False,0.0000,Commerce,Financial,MoreThan15Years
2,True,44,10630.0000,EE,26.7100,Secondary,LessThan3Years,Self-employed,349.4300,Male,Tenant,1200.0000,905.0000,60,Single,True,1.0000,Service,Financial,MoreThan15Years
3,True,42,1500.0000,ES,7.3600,Basic,3To5Years,Employed,449.4700,Male,Other,863.0000,350.0000,60,Cohabitant,True,0.0000,Other,Financial,5To15Years
4,True,34,1595.0000,ES,36.0400,Secondary,LessThan3Years,Unemployed,95.8100,Female,Tenant,697.0000,940.0000,48,Single,True,2.0000,Other,Personal,5To15Years
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
34740,True,38,2000.0000,FI,9.4000,Vocational,3To5Years,Employed,1393.8900,Female,Tenant,2300.0000,830.0000,60,Cohabitant,True,0.0000,Other,Personal,5To15Years
34741,False,37,7440.0000,FI,69.7700,Secondary,MoreThan5Years,Employed,5.5100,Male,Tenant,2172.0000,2488.0900,60,Single,False,0.0000,Service,Financial,MoreThan15Years
34742,False,37,1595.0000,ES,30.1300,Vocational,MoreThan5Years,Employed,67.9200,Male,Tenant,1550.0000,1877.0000,60,Single,False,0.0000,Service,Personal,MoreThan15Years
34743,False,58,3000.0000,EE,25.3500,Vocational,LessThan3Years,Employed,11.2900,Female,Owner,350.0000,359.0000,60,Cohabitant,True,0.0000,Other,Personal,MoreThan15Years


In [27]:
categorical_columns = list(set(df.columns) - set(df._get_numeric_data().columns))
categorical_columns = sorted(categorical_columns)
categorical_columns

['Country',
 'Education',
 'EmploymentDurationCurrentEmployer',
 'EmploymentStatus',
 'Gender',
 'HomeOwnershipType',
 'MaritalStatus',
 'OccupationArea',
 'UseOfLoan',
 'WorkExperience']

In [28]:
target = 'Default'
t_counts = df[target].value_counts()

for c in categorical_columns:
    values = df[c].unique()
    c_counts_positive = df[df[target] == True][c].value_counts().to_dict()
    c_counts_negative = df[df[target] == False][c].value_counts().to_dict()
    woe_c = 'WOE' + c
    print(f'{woe_c}:')
    df[woe_c] = 0
    for v in values:
        woe_v = np.log((c_counts_positive[v] / t_counts[True]) / (c_counts_negative[v] / t_counts[False]))
        print(f'    {v}: {woe_v}')
        df.loc[df[c] == v, woe_c] = woe_v

WOECountry:
    EE: -0.4964489173416248
    ES: 0.9820840087956748
    FI: 0.7346433437112573
    SK: 2.0040101914830024
WOEEducation:
    Secondary: -0.2142185510615049
    Higher: -0.2081000073841618
    Basic: 0.322383860752196
    Vocational: 0.5210364189757211
WOEEmploymentDurationCurrentEmployer:
    MoreThan5Years: -0.026898308419436426
    LessThan3Years: 0.014636667158690404
    3To5Years: 0.025634386879353627
WOEEmploymentStatus:
    Unemployed: 0.46493481741133447
    Self-employed: -0.07319540182838082
    Employed: -0.017054107357713552
WOEGender:
    Female: -0.03062934131370784
    Male: 0.02423330564590897
WOEHomeOwnershipType:
    Owner: -0.2004496647459641
    Tenant: 0.10635183287054537
    Other: 0.20523599773080536
WOEMaritalStatus:
    Cohabitant: -0.10843725466909426
    Single: 0.13549521411489918
WOEOccupationArea:
    Other: 0.11047103307378767
    Commerce: 0.04889913119571511
    Service: -0.0771559067483388
    Industry: -0.03109521870696129
WOEUseOfLoan:
 

In [29]:
df

,Default,Age,AppliedAmount,Country,DebtToIncome,Education,EmploymentDurationCurrentEmployer,EmploymentStatus,FreeCash,Gender,...,WOECountry,WOEEducation,WOEEmploymentDurationCurrentEmployer,WOEEmploymentStatus,WOEGender,WOEHomeOwnershipType,WOEMaritalStatus,WOEOccupationArea,WOEUseOfLoan,WOEWorkExperience
0,True,53,2125.0000,EE,26.2900,Secondary,MoreThan5Years,Unemployed,10.9200,Female,...,-0.4964,-0.2142,-0.0269,0.4649,-0.0306,-0.2004,-0.1084,0.1105,-0.0249,0.0350
1,True,50,3000.0000,EE,30.5800,Higher,MoreThan5Years,Self-employed,78.8000,Female,...,-0.4964,-0.2081,-0.0269,-0.0732,-0.0306,-0.2004,-0.1084,0.0489,-0.0249,0.0350
2,True,44,10630.0000,EE,26.7100,Secondary,LessThan3Years,Self-employed,349.4300,Male,...,-0.4964,-0.2142,0.0146,-0.0732,0.0242,0.1064,0.1355,-0.0772,-0.0249,0.0350
3,True,42,1500.0000,ES,7.3600,Basic,3To5Years,Employed,449.4700,Male,...,0.9821,0.3224,0.0256,-0.0171,0.0242,0.2052,-0.1084,0.1105,-0.0249,-0.0213
4,True,34,1595.0000,ES,36.0400,Secondary,LessThan3Years,Unemployed,95.8100,Female,...,0.9821,-0.2142,0.0146,0.4649,-0.0306,0.1064,0.1355,0.1105,0.0271,-0.0213
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
34740,True,38,2000.0000,FI,9.4000,Vocational,3To5Years,Employed,1393.8900,Female,...,0.7346,0.5210,0.0256,-0.0171,-0.0306,0.1064,-0.1084,0.1105,0.0271,-0.0213
34741,False,37,7440.0000,FI,69.7700,Secondary,MoreThan5Years,Employed,5.5100,Male,...,0.7346,-0.2142,-0.0269,-0.0171,0.0242,0.1064,0.1355,-0.0772,-0.0249,0.0350
34742,False,37,1595.0000,ES,30.1300,Vocational,MoreThan5Years,Employed,67.9200,Male,...,0.9821,0.5210,-0.0269,-0.0171,0.0242,0.1064,0.1355,-0.0772,0.0271,0.0350
34743,False,58,3000.0000,EE,25.3500,Vocational,LessThan3Years,Employed,11.2900,Female,...,-0.4964,0.5210,0.0146,-0.0171,-0.0306,-0.2004,-0.1084,0.1105,0.0271,0.0350


In [30]:
numerical_df = df.drop(columns=categorical_columns)
numerical_df['NewCreditCustomer'] = numerical_df['NewCreditCustomer'].astype(int)
numerical_df['Default'] = numerical_df['Default'].astype(int)
numerical_df

,Default,Age,AppliedAmount,DebtToIncome,FreeCash,IncomeTotal,LiabilitiesTotal,LoanDuration,NewCreditCustomer,NrOfDependants,WOECountry,WOEEducation,WOEEmploymentDurationCurrentEmployer,WOEEmploymentStatus,WOEGender,WOEHomeOwnershipType,WOEMaritalStatus,WOEOccupationArea,WOEUseOfLoan,WOEWorkExperience
0,1,53,2125.0000,26.2900,10.9200,354.0000,485.0900,60,0,0.0000,-0.4964,-0.2142,-0.0269,0.4649,-0.0306,-0.2004,-0.1084,0.1105,-0.0249,0.0350
1,1,50,3000.0000,30.5800,78.8000,900.0000,736.4500,60,0,0.0000,-0.4964,-0.2081,-0.0269,-0.0732,-0.0306,-0.2004,-0.1084,0.0489,-0.0249,0.0350
2,1,44,10630.0000,26.7100,349.4300,1200.0000,905.0000,60,1,1.0000,-0.4964,-0.2142,0.0146,-0.0732,0.0242,0.1064,0.1355,-0.0772,-0.0249,0.0350
3,1,42,1500.0000,7.3600,449.4700,863.0000,350.0000,60,1,0.0000,0.9821,0.3224,0.0256,-0.0171,0.0242,0.2052,-0.1084,0.1105,-0.0249,-0.0213
4,1,34,1595.0000,36.0400,95.8100,697.0000,940.0000,48,1,2.0000,0.9821,-0.2142,0.0146,0.4649,-0.0306,0.1064,0.1355,0.1105,0.0271,-0.0213
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
34740,1,38,2000.0000,9.4000,1393.8900,2300.0000,830.0000,60,1,0.0000,0.7346,0.5210,0.0256,-0.0171,-0.0306,0.1064,-0.1084,0.1105,0.0271,-0.0213
34741,0,37,7440.0000,69.7700,5.5100,2172.0000,2488.0900,60,0,0.0000,0.7346,-0.2142,-0.0269,-0.0171,0.0242,0.1064,0.1355,-0.0772,-0.0249,0.0350
34742,0,37,1595.0000,30.1300,67.9200,1550.0000,1877.0000,60,0,0.0000,0.9821,0.5210,-0.0269,-0.0171,0.0242,0.1064,0.1355,-0.0772,0.0271,0.0350
34743,0,58,3000.0000,25.3500,11.2900,350.0000,359.0000,60,1,0.0000,-0.4964,0.5210,0.0146,-0.0171,-0.0306,-0.2004,-0.1084,0.1105,0.0271,0.0350


In [31]:
numerical_df.to_csv('../data/NumericalLoanData.csv', index=False)